# NSGA2 based pareto-optimal solution for the problem



In [1]:
import numpy as np
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from math import sqrt, acos

# Constants

In [2]:
# Constants
H = 570  # mm

# Evaluation Functions

In [3]:
def calculate_upper_bound_a(L):
    """Calculate the upper bound of 'a' based on 'L'."""
    return sqrt(L**2 + H**2) / 2


def phi(a, L):
    return acos(H / sqrt(H**2 + L**2))


def d_center_of_mass(a, L):
    return L / 2


def h_max(a, L):
    b_prime = L - a
    c = H
    s = (a + b_prime + c) / 2

    inner = s * (s - a) * (s - b_prime) * (s - c)
    if inner < 0:
        return np.inf

    S_heron = sqrt(inner)
    return (2 * S_heron) / b_prime

# Optimization Problem Definition

In [4]:
class OptimizationProblem(ElementwiseProblem):
    def __init__(
        self,
        n_var,
        n_obj,
        n_ieq_constr,
    ):
        super().__init__(
            n_var=n_var,
            n_obj=n_obj,
            n_ieq_constr=n_ieq_constr,
            xl=self.lower_bounds(),
            xu=self.upper_bounds(),
        )

    @staticmethod
    def lower_bounds():
        """Define lower bounds for variables."""
        return np.array([425, 300])

    @staticmethod
    def upper_bounds():
        """Define upper bounds for variables."""
        return np.array([650, calculate_upper_bound_a(650)])

    def _evaluate(self, x, out, *args, **kwargs):
        L, a = x

        # Objective functions
        f1 = phi(a, L)
        f2 = d_center_of_mass(a, L)
        f3 = h_max(a, L)

        # Constraints:
        # Triangular constraint
        triangle_1 = a - L
        triangle_2 = H - (L + a)
        triangle_3 = L - a - H

        # s*(s-a)*(s-b')*(s-c) > 0 constraint
        b_prime = L - a
        c = H
        s = (a + b_prime + c) / 2
        heron_expression = s * (s - a) * (s - b_prime) * (s - c)
        heron_constraint = -heron_expression
        # heron_constraint > 0 means that the triangle is valid

        # Store objectives
        out["F"] = [f1, f2, f3]

        # Store constraints
        out["G"] = [triangle_1, triangle_2, triangle_3, heron_constraint]

# Execution

In [5]:
def init_nsga2(
    n_var,
    n_obj,
    n_ieq_constr,
):
    problem = OptimizationProblem(
        n_var=n_var,
        n_obj=n_obj,
        n_ieq_constr=n_ieq_constr,
    )

    # Define the algorithm
    algorithm = NSGA2(
        pop_size=400,
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True,
    )

    # Perform the optimization
    res = minimize(
        problem,
        algorithm,
        termination=("n_gen", 200),
        seed=1,
        save_history=True,
        verbose=False,
    )

    # Extract the Pareto-optimal solutions
    pareto_solutions = res.X
    pareto_objectives = res.F

    # Print the results
    print("-" * 80)
    print("\nPareto-optimal solutions:")
    for sol, obj in zip(pareto_solutions, pareto_objectives):
        L, a = sol
        f1, f2, f3 = obj
        f1_deg = np.rad2deg(f1)
        b = sqrt(H**2 + L**2) - a
        print(">" * 30)
        print(
            f"Variables:\nL: {L:.2f} mm, a: {a:.2f} mm, b: {b:.2f} mm\n"
            f"Objectives:\nPhi: {f1_deg:.4f} deg, D_center_of_mass: {f2:.2f} mm, H_max: {f3:.2f} mm"
        )
    print("\n" + "-" * 80)

    return pareto_solutions, pareto_objectives

In [6]:
pareto_solutions, pareto_objectives = init_nsga2(
    n_var=2,
    n_obj=3,
    n_ieq_constr=4,
)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

--------------------------------------------------------------------------------

Pareto-optimal solutions:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Variables:
L: 570.00 mm, a: 300.00 mm, b: 506.10 mm
Objectives:
Phi: 45.0000 deg, D_center_of_mass: 285.00 mm, H_max: 0.03 mm
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Variables:
L: 570.00 mm, a: 300.00 mm, b: 506.10 mm
Objectives:
Phi: 45.0000 deg, D_center_of_mass: 285.00 mm, H_max: 0.03 mm
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Variables:
L: 570.00 mm, a: 300.01 mm, b: 506.09 mm
Objectives:
Phi: 45.0000 deg, D_center_of_mass: 285.00 mm, H_max: 0.03 mm

--------------------------------------------------------------------------------
